# Customer Churn Analysis
Churn rate, when applied to a customer base, refers to the proportion of contractual customers or subscribers who leave a supplier during a given time period. It is a possible indicator of customer dissatisfaction, cheaper and/or better offers from the competition, more successful sales and/or marketing by the competition, or reasons having to do with the customer life cycle.

Churn is closely related to the concept of **average customer lifetime**. 
- For example, an annual churn rate of 25 percent implies an average customer life of four years. An annual churn rate of 33 percent implies an average customer life of three years. 
- The churn rate can be minimized by creating barriers which discourage customers to change suppliers (contractual binding periods, use of proprietary technology, value-added services, unique business models, etc.), or through retention activities such as loyalty programs. 
- It is possible to overstate the churn rate, as when a consumer drops the service but then restarts it within the same year. Thus, a clear distinction needs to be made between **"gross churn"**, the total number of absolute disconnections, and **"net churn"**, the overall loss of subscribers or members. The difference between the two measures is the number of **new subscribers** or members that have joined during the same period. 
- Suppliers may find that if they offer a loss-leader "introductory special", it can lead to a higher churn rate and subscriber abuse, as some subscribers will sign on, let the service lapse, then sign on again to take continuous advantage of current specials. https://en.wikipedia.org/wiki/Churn_rate

In [4]:
%%capture

# Get our favorite packages,cufflinks, from PyPI
import numpy as np
import pandas as pd

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

In [5]:
# Load our favorite visualization library
import os
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Sign into Plotly with masked, encrypted API key
#myPlotlyKey = os.environ['SECRET_ENV_BRETTS_PLOTLY_KEY']
py.sign_in(username='bella931210',api_key='pPVioCSsMu8HK9HCjM8d')

In [6]:
# Load data
churnDF = pd.read_csv('churn.csv', delimiter=',')
churnDF.shape
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=False, value='Retain')
churnDF["Churn"] = churnDF["Churn"].replace(to_replace=True, value='Churn')

In [7]:
# fraction of rows
# here you get 7% of the rows
churnDFs = churnDF.sample(frac=0.07) # Sample for speedy viz
churnDF.head(10)

State  Account Length  Area Code     Phone Int'l Plan VMail Plan  \
0    KS             128        415  382-4657         no        yes   
1    OH             107        415  371-7191         no        yes   
2    NJ             137        415  358-1921         no         no   
3    OH              84        408  375-9999        yes         no   
4    OK              75        415  330-6626        yes         no   
5    AL             118        510  391-8027        yes         no   
6    MA             121        510  355-9993         no        yes   
7    MO             147        415  329-9001        yes         no   
8    LA             117        408  335-4719         no         no   
9    WV             141        415  330-8173        yes        yes   

   VMail Message  Day Mins  Day Calls  Day Charge   ...    Eve Calls  \
0             25     265.1        110       45.07   ...           99   
1             26     161.6        123       27.47   ...          103   
2              0     243.4        114       41.38   ...          110   
3              0     299.4         71       50.90   ...           88   
4              0     166.7        113       28.34   ...          122   
5              0     223.4         98       37.98   ...          101   
6             24     218.2         88       37.09   ...          108   
7              0     157.0         79       26.69   ...           94   
8              0     184.5         97       31.37   ...           80   
9             37     258.6         84       43.96   ...          111   

   Eve Charge  Night Mins  Night Calls  Night Charge  Intl Mins  Intl Calls  \
0       16.78       244.7           91         11.01       10.0           3   
1       16.62       254.4          103         11.45       13.7           3   
2       10.30       162.6          104          7.32       12.2           5   
3        5.26       196.9           89          8.86        6.6           7   
4       12.61       186.9          121          8.41       10.1           3   
5       18.75       203.9          118          9.18        6.3           6   
6       29.62       212.6          118          9.57        7.5           7   
7        8.76       211.8           96          9.53        7.1           6   
8       29.89       215.8           90          9.71        8.7           4   
9       18.87       326.4           97         14.69       11.2           5   

   Intl Charge  CustServ Calls   Churn  
0         2.70               1  Retain  
1         3.70               1  Retain  
2         3.29               0  Retain  
3         1.78               2  Retain  
4         2.73               3  Retain  
5         1.70               0  Retain  
6         2.03               3  Retain  
7         1.92               0  Retain  
8         2.35               1  Retain  
9         3.02               0  Retain  

[10 rows x 21 columns]

In [8]:
# separate the calls data for plotting

churnDFs = churnDFs[['Account Length','Day Calls','Eve Calls','CustServ Calls','Churn']]

# Create scatter plot matrix of call data
splom = ff.create_scatterplotmatrix(churnDFs, diag='histogram', index='Churn',  
                                  colormap= dict(
                                      Churn = '#9CBEF1',
                                      Retain = '#04367F'
                                      ),
                                  colormap_type='cat',
                                  height=560, width=650,
                                  size=4, marker=dict(symbol='circle'))
py.iplot(splom)

In [9]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function
h2o.init(nthreads=1, max_mem_size=2)
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


--------------------------  ----------------------------------------
H2O cluster uptime:         53 secs
H2O cluster timezone:       Australia/Sydney
H2O data parsing timezone:  UTC
H2O cluster version:        3.22.0.2
H2O cluster version age:    4 days
H2O cluster name:           H2O_from_python_libaisun_kk2gji
H2O cluster total nodes:    1
H2O cluster free memory:    1.770 Gb
H2O cluster total cores:    4
H2O cluster allowed cores:  1
H2O cluster status:         locked, healthy
H2O connection url:         http://localhost:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4
Python version:             3.6.5 final
--------------------------  ----------------------------------------

In [10]:
# Split data into training and testing frames

from sklearn import cross_validation
from sklearn.model_selection import train_test_split
# stratify
training, testing = train_test_split(churnDF, train_size=0.8, stratify=churnDF["Churn"], random_state=9)
train = h2o.H2OFrame(python_obj=training).drop("State")
test = h2o.H2OFrame(python_obj=testing).drop("State")

# Set predictor and response variables
y = "Churn"
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Super Learner
The super learner is a prediction method designed to find the **optimal combination of a collection of prediction algorithms**. The super learner algorithm finds the **combination of algorithms minimizing the cross-validated risk**. The super learner framework is9 built in the theory of cross-validation and allows for a general class of prediction algorithms to be considered for the ensemble. http://biostats.bepress.com/ucbbiostat/paper266/ (Polley & Van der Laan, 2010)

In [11]:
# Reset variables
# del allModels, gridGBM, gridRF, grids, dfGridGBM, dfGridRF, ensemble


In [12]:
# Reset variables
# del SuperModel, BestModel, Model3, Model4, Model5, Model6, Model7, Model8, Model9, Model10

In [11]:
%%time
# GBM hyperparameters 
nfolds = 5
gbm_hyper_params = {"learn_rate":[0.075, 0.1], "nbins":[10,15,20],"ntrees": [20,30,40], "max_depth": [5,7,9], "sample_rate": [0.75, 0.8, 0.85, 0.9]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 6}
gridGBM = H2OGridSearch(model=H2OGradientBoostingEstimator(balance_classes=True, seed=123, nfolds=nfolds, fold_assignment="Modulo", keep_cross_validation_predictions=True),
                     hyper_params=gbm_hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
# h2o does not have XGBoost
gridGBM.train(x=x, y=y, training_frame=train)
         

    
# Random Forest hyperparameters
rf_hyper_params = {"mtries":[12,15,18],"nbins":[10,20,30], "ntrees": [25,50,75], "max_depth": [5,7], "sample_rate": [0.75, 0.8, 0.85, 0.9]}
gridRF = H2OGridSearch(model=H2ORandomForestEstimator(balance_classes=True, seed=123, nfolds=nfolds, fold_assignment="Modulo", keep_cross_validation_predictions=True),                     
                      hyper_params=rf_hyper_params,
                      search_criteria=search_criteria,
                      grid_id="rf_grid_binomial")
gridRF.train(x=x, y=y, training_frame=train)


# List the GBMs and Random Forests that we wish to ensemble
grids = gridGBM.model_ids + gridRF.model_ids


# Train the super learner
ensemble = H2OStackedEnsembleEstimator(model_id="GBM-RF-ensemble", base_models=grids, training_frame=train, validation_frame=test)
ensemble.train(x=x, y=y, training_frame=train)


# Evaluate ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)


# Compare the super learner to the base learners. First, combine all the base models into a single list, sorted by auc.
# Area Under the Curve (AUC)
dfGridGBM = pd.DataFrame(data=gridGBM.get_grid(sort_by="auc", decreasing=True).sorted_metric_table())
dfGridRF = pd.DataFrame(data=gridRF.get_grid(sort_by="auc", decreasing=True).sorted_metric_table())
allModels = dfGridGBM.append(dfGridRF)
allModels['auc'] = allModels['auc'].astype('float64')
allModels.sort_values(by="auc", ascending=False, inplace=True)
allModels = allModels.reset_index()

# base learner
baselearner_best_name = allModels.loc[0,'model_ids']
baselearner_best_auc = allModels.loc[0,'auc']

# Best stacked model auc
stack_auc_test = perf_stack_test.auc()


print("Best Base-learner Test AUC: " + str(baselearner_best_auc))
print("Ensemble Test AUC: " + str(stack_auc_test))

gbm Grid Build progress: |████████████████████████████████████████████████| 100%
drf Grid Build progress: |████████████████████████████████████████████████| 100%
stackedensemble Model Build progress: |███████████████████████████████████| 100%
Best Base-learner Test AUC: 0.9009061676211254
Ensemble Test AUC: 0.9439500813890397
CPU times: user 2.19 s, sys: 232 ms, total: 2.42 s
Wall time: 57.6 s


## Leaderboard

In [13]:
allModels.sort_values(by='auc',ascending=False )

index         auc learn_rate max_depth                  model_ids mtries  \
0       0    0.900906        0.1         5  gbm_grid_binomial_model_6    NaN   
1       1    0.898035        0.1         9  gbm_grid_binomial_model_3    NaN   
2       2    0.897202      0.075         9  gbm_grid_binomial_model_1    NaN   
3       3    0.895326      0.075         9  gbm_grid_binomial_model_4    NaN   
4       4    0.894264      0.075         5  gbm_grid_binomial_model_5    NaN   
5       5    0.893466      0.075         9  gbm_grid_binomial_model_2    NaN   
6       0    0.885327        NaN         5   rf_grid_binomial_model_4     15   
7       1    0.883745        NaN         5   rf_grid_binomial_model_2     15   
8       2    0.883726        NaN         5   rf_grid_binomial_model_3     18   
9       3    0.883642        NaN         5   rf_grid_binomial_model_1     12   
10      4    0.881974        NaN         7   rf_grid_binomial_model_5     12   
11      5    0.880061        NaN         5   rf_grid_binomial_model_6     18   

   nbins ntrees sample_rate  
0     15     40        0.75  
1     20     30         0.9  
2     20     30        0.85  
3     20     20        0.85  
4     15     20         0.8  
5     20     20         0.9  
6     10     25        0.85  
7     20     50         0.9  
8     10     50         0.8  
9     20     50        0.85  
10    10     75         0.9  
11    30     25         0.9

# Variable Importances
Below we plot variable importances as reported by the best performing algo in the ensemble.

In [14]:
best = h2o.get_model(baselearner_best_name)

importances = best.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

# Super Model vs the Base models
This plot shows the ROC curves for the Super Model, the Best Base Model, and 9 next best models in the ensemble. 

In [15]:
SuperModel = np.array(ensemble.roc(valid=True))
BestModel = np.array(h2o.get_model(baselearner_best_name).roc(xval=True))
Model2 = np.array(h2o.get_model(allModels.loc[1,'model_ids']).roc(xval=True))
Model3 = np.array(h2o.get_model(allModels.loc[2,'model_ids']).roc(xval=True))
Model4 = np.array(h2o.get_model(allModels.loc[3,'model_ids']).roc(xval=True))
Model5 = np.array(h2o.get_model(allModels.loc[4,'model_ids']).roc(xval=True))
Model6 = np.array(h2o.get_model(allModels.loc[5,'model_ids']).roc(xval=True))
Model7 = np.array(h2o.get_model(allModels.loc[6,'model_ids']).roc(xval=True))
Model8 = np.array(h2o.get_model(allModels.loc[7,'model_ids']).roc(xval=True))
Model9 = np.array(h2o.get_model(allModels.loc[8,'model_ids']).roc(xval=True))
Model10 = np.array(h2o.get_model(allModels.loc[9,'model_ids']).roc(xval=True))



layout = go.Layout(autosize=False, width=725, height=575,  xaxis=dict(title='False Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')), 
                                                           yaxis=dict(title='True Positive Rate', titlefont=dict(family='Arial, sans-serif', size=15, color='grey')))

SuperModelTrace = go.Scatter(x = SuperModel[0],y = SuperModel[1], mode = 'lines', name = 'Super Model', line = dict(color = ('rgb(26, 58, 126)'), width = 3))
BestModelTrace = go.Scatter(x = BestModel[0],y = BestModel[1], mode = 'lines', name = 'Best Base Model', line = dict(color = ('rgb(135, 160, 216)'), width = 3))
Model2Trace = go.Scatter(x = Model2[0], y = Model2[1], mode = 'lines', name = 'Model 2', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model3Trace = go.Scatter(x = Model3[0], y = Model3[1], mode = 'lines', name = 'Model 3', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model4Trace = go.Scatter(x = Model4[0], y = Model4[1], mode = 'lines', name = 'Model 4', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model5Trace = go.Scatter(x = Model5[0], y = Model5[1], mode = 'lines', name = 'Model 5', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model6Trace = go.Scatter(x = Model6[0], y = Model6[1], mode = 'lines', name = 'Model 6', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model7Trace = go.Scatter(x = Model7[0], y = Model7[1], mode = 'lines', name = 'Model 7', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model8Trace = go.Scatter(x = Model8[0], y = Model8[1], mode = 'lines', name = 'Model 8', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model9Trace = go.Scatter(x = Model9[0], y = Model9[1], mode = 'lines', name = 'Model 9', line = dict(color = ('rgb(156, 190, 241)'), width = 1))
Model10Trace = go.Scatter(x = Model10[0], y = Model10[1], mode = 'lines', name = 'Model 10', line = dict(color = ('rgb(156, 190, 241)'), width = 1))

traceChanceLine = go.Scatter(x = [0,1], y = [0,1], mode = 'lines+markers', name = 'chance', line = dict(color = ('rgb(136, 140, 150)'), width = 4, dash = 'dash'))

fig = go.Figure(data=[SuperModelTrace,BestModelTrace,Model2Trace,Model3Trace,Model4Trace,Model5Trace,Model7Trace,Model8Trace,Model9Trace,Model10Trace,traceChanceLine], layout=layout)


py.iplot(fig)


# Confusion Matrix

In [16]:
cm = perf_stack_test.confusion_matrix()
cm = cm.table.as_data_frame()
cm
confusionMatrix = ff.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)


# Business Impact Matrix

Weighting Predictions With a Dollar Value
-   Correctly predicting retain: `+$5`
-   Correctly predicting churn: `+$75`
-   Incorrectly predicting retain: `-$150`
-   Incorrectly predicting churn: `-$1.5`

    

In [18]:
CorrectPredictChurn = cm.loc[0,'Churn']
CorrectPredictChurnImpact = 75
cm1 = CorrectPredictChurn*CorrectPredictChurnImpact

IncorrectPredictChurn = cm.loc[1,'Churn']
IncorrectPredictChurnImpact = -5
cm2 = IncorrectPredictChurn*IncorrectPredictChurnImpact

IncorrectPredictRetain = cm.loc[0,'Retain']
IncorrectPredictRetainImpact = -150
cm3 = IncorrectPredictRetain*IncorrectPredictRetainImpact

CorrectPredictRetain = cm.loc[0,'Retain']
CorrectPredictRetainImpact = 5
cm4 = IncorrectPredictRetain*CorrectPredictRetainImpact


data_matrix = [['Business Impact', '($) Predicted Churn', '($) Predicted Retain', '($) Total'],
               ['($) Actual Churn', cm1, cm3, '' ],
               ['($) Actual Retain', cm2, cm4, ''],
               ['($) Total', cm1+cm2, cm3+cm4, cm1+cm2+cm3+cm4]]

impactMatrix = ff.create_table(data_matrix, height_constant=20)
impactMatrix.layout.height=300
impactMatrix.layout.width=800
impactMatrix.layout.font.size=17
py.iplot(impactMatrix)

In [19]:
print("Best learner AUC: " + str(baselearner_best_auc))

Best learner AUC: 0.9009061676211254


In [20]:
print("Super Model AUC: " + str(stack_auc_test))

Super Model AUC: 0.9439500813890397


In [21]:
print("Total customers evaluated: 534")

Total customers evaluated: 534


In [22]:
print("Total value created by the model: $" + str(cm1+cm2+cm3+cm4))

Total value created by the model: $2415.0


In [23]:
print("Total value per customer: $" +str(round(((cm1+cm2+cm3+cm4)/534),3)))

Total value per customer: $4.522


In [24]:
# Save the Model
h2o.save_model(model=ensemble, force=True)
LoadedEnsemble = h2o.load_model(path='GBM-RF-ensemble')